# Construcción del `VIM` - El Volatility Index del Merval 
La idea del proyecto es comenzar a definir un índice similar al `VIX` pero para el mercado de acciones argentino.
A este índice lo llamaremos `VIM`(Volatility Index Merval).
Como primera iteración utilizaremos las opciones de Grupo Financiero Galicia `GGAL`, luego iremos agregando más acciones para hacerlo más representativo. 
Con contar con las opciones de `GGAL`, `YPF`, `BMA`, `PAMP`, `TGS` y alguna más será suficiente. La idea es tomar los 5 o 6 ADRs más líquidos y con una buena cantidad de opciones.

>*"CBOE VIX - White Paper: Cbow Volatility Index"*

El `VIX` se calcula de la siguiente forma:

![](vix_formula.png)

## Primera Iteración: Cálculo de la volatilidad implícita de una opción

Se define *volatilidad* como la desviación estándar de los cambios porcentuales de los precios.

Usualmente utilizamos dos medidas de la volatilidad:
1. **Volatilidad Histórica:** Se calcula con precios históricos.
2. **Volatilidad Implícita:** Se calcula a partir de la prima de las opciones. Nos da una estimación del movimiento probable futuro de los precios. Es decir, se obtiene al despejar la volatilidad en las fórmulas con las que valuamos opciones.

Nos vamos a centrar en la **Volatilidad Implícita**
La base del cálculo es con Strikes (precio de ejercicio) que se encuentren "at the money" para vencimientos a 30 días de la expiración.


In [1]:

from scipy.stats import norm
from math import log, sqrt, exp

def calculate_iv(C, S, K, T, r):
    # Estimación inicial de la volatilidad
    sigma = 0.2  
    tolerance = 0.0001
    max_iterations = 1000

    for i in range(max_iterations):
        d1 = (log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
        d2 = d1 - sigma * sqrt(T)
        
        # Precio de la opción call con la volatilidad actual
        option_price = S * norm.cdf(d1) - K * exp(-r * T) * norm.cdf(d2)
        
        # Vega (derivada del precio de la opción con respecto a la volatilidad)
        vega = S * norm.pdf(d1) * sqrt(T)
        
        # Diferencia entre el precio de la opción calculado y el real
        price_difference = option_price - C
        
        # Verificar si la diferencia está dentro del rango de tolerancia
        if abs(price_difference) < tolerance:
            return sigma
        
        # Actualizar la estimación de la volatilidad
        sigma = sigma - price_difference / vega
    
    raise Exception("No se encontró la volatilidad dentro de la tolerancia especificada")

# Parámetros dados
C = 118.05
S = 400
K = 300
T = 5 / 12
r = 0.10

# Calcular la volatilidad implícita
volatilidad_implicita = calculate_iv(C, S, K, T, r)
print("Volatilidad Implícita:", volatilidad_implicita)

Volatilidad Implícita: 0.4394726511734419
